In [1]:
import tensorflow as tf

from network import network_big
import coremltools

/Users/hyunkim/.pyenv/versions/3.6.5/envs/lowerTF/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
h5_path = "./trained_models/widerScale/2862-0.05.h5"
mlmodel_path = "./mlmodel/WiderScaleSOTA_outImage.mlmodel"
model = network_big((256,256,3), android=False, train=False)
model.load_weights(h5_path, by_name=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 8)  224         input_1[0][0]                    
__________________________________________________________________________________________________
p_re_lu (PReLU)                 (None, 256, 256, 8)  8           conv2d[0][0]                     
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 256, 256, 8)  144         p_re_lu[0][0]                    
______________________________________________________________________________________________

In [6]:
ml_model = coremltools.converters.keras.convert(model, 
                                                input_names='image', 
                                                image_input_names='image', 
                                                red_bias = -(123.68) / 58.393,
                                                green_bias = -(116.779) / 57.12, 
                                                blue_bias = -(103.939) / 57.375,
                                                image_scale = 1.0 / 57.63)

0 : input_1, <keras.engine.input_layer.InputLayer object at 0x14f6ca2e8>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x14f6cab00>
2 : p_re_lu_1, <keras.layers.advanced_activations.PReLU object at 0x14f626208>
3 : separable_conv2d_1, <keras.layers.convolutional.SeparableConv2D object at 0x14f6cae10>
4 : p_re_lu_2, <keras.layers.advanced_activations.PReLU object at 0x14fe23358>
5 : separable_conv2d_2, <keras.layers.convolutional.SeparableConv2D object at 0x14fe45240>
6 : add_1, <keras.layers.merge.Add object at 0x14f7a1a90>
7 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x14f2d1358>
8 : p_re_lu_3, <keras.layers.advanced_activations.PReLU object at 0x14f35fc88>
9 : separable_conv2d_3, <keras.layers.convolutional.SeparableConv2D object at 0x14f35f978>
10 : p_re_lu_4, <keras.layers.advanced_activations.PReLU object at 0x14f10c240>
11 : separable_conv2d_4, <keras.layers.convolutional.SeparableConv2D object at 0x14f10f550>
12 : add_2, <keras.layers.merge.Add object 

106 : add_22, <keras.layers.merge.Add object at 0x14f692e80>
107 : p_re_lu_39, <keras.layers.advanced_activations.PReLU object at 0x14f699ef0>
108 : separable_conv2d_39, <keras.layers.convolutional.SeparableConv2D object at 0x14f847710>
109 : p_re_lu_40, <keras.layers.advanced_activations.PReLU object at 0x14f84f550>
110 : separable_conv2d_40, <keras.layers.convolutional.SeparableConv2D object at 0x14f8582b0>
111 : add_23, <keras.layers.merge.Add object at 0x14faf96d8>
112 : conv2d_transpose_4, <keras.layers.convolutional.Conv2DTranspose object at 0x14fb13c18>
113 : add_24, <keras.layers.merge.Add object at 0x14fc20e48>
114 : p_re_lu_41, <keras.layers.advanced_activations.PReLU object at 0x14fc43c50>
115 : separable_conv2d_41, <keras.layers.convolutional.SeparableConv2D object at 0x14fc43b70>
116 : p_re_lu_42, <keras.layers.advanced_activations.PReLU object at 0x14fc51c88>
117 : separable_conv2d_42, <keras.layers.convolutional.SeparableConv2D object at 0x14fd9d9e8>
118 : add_25, <keras

In [9]:
ml_model.save(mlmodel_path)